In [28]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_AWZDTJC7ELLLWKQJM7XSDOUT-QMTVJN4LDJALI4EQ")

# constansts

START_WORD_LENGTH = 6

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 8

NUM_STEPS = 8
MIN_STEPS = 5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Disconnecting from previous connection first...
Connecting to wss://anvil.works/uplink
Anvil websocket closed (code 1000, reason=b'')
Anvil websocket open
Connected to "Debug for azrafe7@gmail.com" as SERVER


In [29]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
dictionaryFileName = 'assets/Ita.txt'

    
def normalizeWord(word):
    return word.upper()
    
LOAD_FROM_URL = True

t0 = time.time()

lines = []
if LOAD_FROM_URL:
    baseUrl = 'https://azrafe7.github.io/'
    f = requests.get(baseUrl + dictionaryFileName)
    lines = f.text.splitlines()
else:
    with open(dictionaryFileName) as f:
        lines = f.read().splitlines()

filteredWords = []
for line in lines:
    length = len(line)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH:
        filteredWords.append(normalizeWord(line))

print("Dictionary loaded in {0:.2f}s".format(time.time() - t0))

print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

Dictionary loaded in 0.41s
202685/710993
['ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACISTA', 'ABACISTI', 'ABACO', 'ABADESSA', 'ABADESSE']


In [30]:


def wordsOfLength(length=5):
    return [word for word in filteredWords if len(word) == length]

len(list(wordsOfLength(START_WORD_LENGTH)))


23552

In [31]:

def isAnagram(sourceWord, targetWord):
    if len(sourceWord) != len(targetWord):
        return False
    
    res = True
    
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    #print(targetCharSet)
    for i, ch in enumerate(sourceWord):
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
        else:
            res = False
            break
    
    return res
    
        
assert(isAnagram('', ''))
assert(not isAnagram('corto', 'trOca'))
assert(not isAnagram('c', ''))
assert(isAnagram('cortao', 'trocao'))

srcWord = 'ostra'
dstWord ='motri'

print('isAnagram("{0}", "{1}")'.format(srcWord, dstWord))
print(isAnagram(srcWord, dstWord))

isAnagram("ostra", "motri")
False


In [32]:
def isAnagramEx(sourceWord, targetWord, maxDiffLength=0, strict=False):
    if maxDiffLength == 0:
        return isAnagram(sourceWord, targetWord)
    else:
        wordDiffLength = abs(len(sourceWord) - len(targetWord))
        if wordDiffLength > maxDiffLength:
            return False
        
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    if len(sourceCharSet) > len(targetCharSet):
        sourceCharSet, targetCharSet = targetCharSet, sourceCharSet
        
    #print(targetCharSet, sourceCharSet)
    for ch in sourceCharSet:
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
    
    #print(targetCharSet)
    targetCharSetLength = len(targetCharSet)
    #print(wordDiffLength, targetCharSet)
    isLooseAnagram = (targetCharSetLength <= maxDiffLength)
    res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and wordDiffLength == maxDiffLength)
    return res
    
        
maxDiffLength=2

assert(isAnagramEx('', ''))
assert(not isAnagramEx('corto', 'trOca'))
assert(not isAnagramEx('c', ''))
assert(isAnagramEx('cortao', 'trocao'))
assert(isAnagramEx('cortao', 'trocaox', 1))
assert(not isAnagramEx('strappo', 'posta', 1))
assert(isAnagramEx('strappo', 'posta', 2))
assert(isAnagramEx('strappo', 'posta', 3))
assert(not isAnagramEx('strappo', 'posta', 3, strict=True))
assert(isAnagramEx('strappo', 'posta', 2, strict=True))
assert(isAnagramEx('LEVI', 'ABILE', 2,))
assert(not isAnagramEx('TUFFAI', 'ABACA', 2,))


srcWord = 'straccio'
dstWord ='crosta'

print('isAnagramEx("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx(srcWord, dstWord, maxDiffLength=maxDiffLength))


isAnagramEx("straccio", "crosta", 2)
True


In [33]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
NodeCount = 0
WordCount = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class TrieNode:
    def __init__(self):
        self.word = None
        self.children = {}

        global NodeCount
        NodeCount += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children: 
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word

# read dictionary file into a trie
start = time.time()

trie = TrieNode()
for word in filteredWords:
    WordCount += 1
    trie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print("Read %d words into %d nodes" % (WordCount, NodeCount))

# The search function returns a list of all words that are less than the given
# maximum distance from the target word
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow, 
            results, maxCost )

    return results

# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:                
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.word, currentRow[-1] ) )

    # if any entries in the row are less than the maximum cost, then 
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow, 
                results, maxCost )

start = time.time()
results = search( TARGET, MAX_COST )
end = time.time()

#for result in results: print(result)

print("Search took %g s" % (end - start))



Build took 1.78132 s
Read 202685 words into 327088 nodes
Search took 0.449069 s


In [34]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

start = time.time()
results = search(startWord, 3)
end = time.time()

print("Search took %g s" % (end - start))

validEndWord = random.choice(results)

print("END WORD:", validEndWord)


START WORD: SCOZZATO
Search took 0.190583 s
END WORD: ('SMOTTATO', 3)


In [35]:
def findAnagramsEx(wordList, word, maxDiffLength=1, strict=False):
    results = []
    wordLength = len(word)
    for entry in wordList:
        entryLength = len(entry)
        if word == entry or entryLength < wordLength - maxDiffLength or entryLength > wordLength + maxDiffLength:
            continue
        if isAnagramEx(entry, word, maxDiffLength=maxDiffLength, strict=strict):
            diffLength = abs(len(entry) - len(word))
            results.append((entry, diffLength))
    
    return results


In [36]:
import random

newStartWord = random.choice(filteredWords)
print("START WORD:", startWord)

anagramsDict = {0:[], 1:[], 2:[]}
randomFilteredWords = filteredWords[:]
random.shuffle(filteredWords[:])
randomFilteredWords = sorted(randomFilteredWords)

anagrams = findAnagramsEx(randomFilteredWords, newStartWord, maxDiffLength=1, strict=False)
for anagram in anagrams:
    anagramsDict[anagram[1]].append(anagram[0])
    
print(anagrams)
#print(anagramsDict[0][:10])

START WORD: SCOZZATO
[('APEIRON', 0), ('APIRANO', 0), ('APIRENO', 0), ('APPINZO', 0), ('ARPIONA', 0), ('ARPIONE', 0), ('ARPIONI', 0), ('ARPIONO', 0), ('ARPONAI', 0), ('ARPONI', 1), ('ARRONZI', 0), ('ASPRINO', 0), ('AZIONAR', 0), ('BRONZAI', 0), ('CAPRINO', 0), ('CAPRONI', 0), ('CARPINO', 0), ('CARPONI', 0), ('CIARPON', 0), ('EPIZONA', 0), ('ERZIANO', 0), ('GARZINO', 0), ('GARZONI', 0), ('GIRONZA', 0), ('GRAZINO', 0), ('INASPRO', 0), ('INFORZA', 0), ('MARPION', 0), ('NAZIREO', 0), ('OPERINA', 0), ('OPINAR', 1), ('OPINARE', 0), ('OPINERA', 0), ('OPUNZIA', 0), ('OPZIONA', 0), ('OPZIONAR', 1), ('ORZANTI', 0), ('OZIARNE', 0), ('PADRINO', 0), ('PADRONI', 0), ('PANARIO', 0), ('PANDORI', 0), ('PANIZZO', 0), ('PANZONI', 0), ('PARAZONI', 1), ('PARIANO', 0), ('PARINO', 1), ('PARLINO', 0), ('PARTONI', 0), ('PATRONI', 0), ('PAZZINO', 0), ('PIANERO', 0), ('PIANORI', 0), ('PIANORO', 0), ('PIARONO', 0), ('PIGNORA', 0), ('PIGRONA', 0), ('PILONAR', 0), ('PINZALO', 0), ('PINZANO', 0), ('PINZAR', 1), ('PI

In [37]:
print(anagramsDict[1])

['ARPONI', 'OPINAR', 'OPZIONAR', 'PARAZONI', 'PARINO', 'PINZAR', 'PINZARLO', 'PIRANO', 'PONZAI', 'PONZAR', 'PONZARCI', 'PONZARLI', 'PONZARMI', 'PONZARSI', 'PONZARTI', 'PONZARVI', 'PONZERAI', 'PORZIONA', 'PRANZI', 'PRANZINO', 'PRANZO', 'PRANZONI', 'PRONAI', 'PROZIA', 'RAPINO', 'RAPONI', 'RIPONZAI', 'RIPONZAR', 'RIPRANZO', 'RONZAI', 'ZARINO', 'ZONARI']


In [38]:
import time

def generateChain(quiet=False, startWord=None, strict=False):
    startWords = wordsOfLength(START_WORD_LENGTH)
    
    restart = True
    
    while restart:
        restart = False
        startWord = random.choice(startWords)
        print("START WORD:", startWord)
    
        steps = [startWord]
    
        prevStep = (startWord, 0)
        #self.info.text = ""
        for i in range(6):
            choice = random.choice(["anagram", "levenshtein"])
            if choice == "anagram":
                if not quiet: print("Finding anagrams...", end="")
                startTime = time.time()
                anagrams = findAnagramsEx(filteredWords, prevStep[0], maxDiffLength=1, strict=strict)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(anagrams) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(anagrams))
                    break
            else:
                if not quiet: print("Finding levenshtein...", end="")
                startTime = time.time()
                results = search(prevStep[0], 1)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(results) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(results))
                    break
                    
            done = False
            while not done:
                if choice == "anagram":
                    nextStep = random.choice(anagrams)
                else:
                    nextStep = random.choice(results)
    
                if nextStep[0] not in steps:
                    steps.append(nextStep[0])
                    done = True
    
            resultsCount = len(results) if choice == "levenshtein" else len(anagrams)
            if not quiet: print(nextStep, resultsCount, choice)
            prevStep = nextStep
    
        print("restart", restart)
        print()
    
    endWord = steps[-1]
    print("END WORD:", endWord)  
    return steps

t0 = time.time()
steps = generateChain(startWord=None, strict=True)
print("Elapsed {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

START WORD: LIBBRA
Finding anagrams... (0.38)
('LABRI', 1) 16 anagram
Finding anagrams... (0.16)
('ALBI', 1) 42 anagram
Finding levenshtein... (0.01)
('ALAI', 1) 21 levenshtein
Finding levenshtein... (0.01)
('ALDI', 1) 28 levenshtein
Finding levenshtein... (0.01)
('ALMI', 1) 20 levenshtein
Finding levenshtein... (0.01)
('ALSI', 1) 23 levenshtein
restart False

END WORD: ALSI
Elapsed 0.63s


In [39]:
def isInDictionary(word):
    results = search(word, 0)
    return len(results) > 0

class WordStatus():
  UNCHECKED = 0
  VALID = 1
  INVALID = 2
  NOT_IN_DICT = 3

def checkValidChain(startWord, endWord, wordChain):
    prevWord = startWord
    isOk = True
    statuses = []
    for word in wordChain + [endWord]:
        inDictionary = isInDictionary(word)
        results = [t[0] for t in search(prevWord, 1)]
        anagrams = [t[0] for t in findAnagramsEx(filteredWords, prevWord, maxDiffLength=1, strict=False)]
        isWordOk = inDictionary and (word in results or word in anagrams) 
        if not inDictionary:
            print("✖️", word)
            statuses.append(WordStatus.NOT_IN_DICT)
        else:
            print("✔️" if isWordOk else "❌", word)
            statuses.append(WordStatus.VALID if isWordOk else WordStatus.INVALID)
        
        if not isWordOk:
            isOk = False
        prevWord = word

    return isOk, statuses
            
print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: LIBBRA
END WORD  : ALSI


In [40]:
def displayCheckValidChain(startWord, endWord, wordChain):
    print()
    print(startWord)
    ok, statuses = checkValidChain(startWord, endWord, wordChain)
    print(endWord)

    print()
    print("👍 WON" if ok else "👎 LOST")

    print()


In [41]:
# regenerate start/end words
t0 = time.time()
steps = generateChain(quiet=True, startWord='piccolo')
print("\nGenerate {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: FUSONE
levenshtein 7
restart True

START WORD: VANUME
levenshtein 2
restart True

START WORD: VIVACE
levenshtein 8
restart True

START WORD: ANGELI
levenshtein 6
restart True

START WORD: GAMETI
restart False

END WORD: ESENTALI

Generate 1.83s
START WORD: GAMETI
END WORD  : ESENTALI


In [42]:
wordChain = [
    'pantera',
    'pentola',
    'pantofola',
    'osteria',
    'ristorai',
    'piantala',
]
wordChain = [w.strip().upper() for w in wordChain]
#print("GUESS WORD CHAIN", wordChain)

t0 = time.time()
displayCheckValidChain(startWord, endWord, wordChain)
print("Validate {0:.2f}s".format(time.time() - t0))


GAMETI
❌ PANTERA
❌ PENTOLA
✖️ PANTOFOLA
❌ OSTERIA
❌ RISTORAI
❌ PIANTALA
❌ ESENTALI
ESENTALI

👎 LOST

Validate 2.48s


In [43]:
# STEPS

print("COMPUTED WORD CHAIN", steps)

COMPUTED WORD CHAIN ['GAMETI', 'GAMETE', 'MESATE', 'MESTATE', 'MESTANTE', 'ESENTAMI', 'ESENTALI']


In [44]:
# find in dictionary

findWord = 'muovi'

search(findWord.strip().upper(), 0)


[('MUOVI', 0)]

In [45]:
print([hex(ord(ch)) for ch in "✖️"])
print([hex(ord(ch)) for ch in "✔️"])
print([hex(ord(ch)) for ch in "❌"])
"👍"
hex(ord("👎"))

['0x2716', '0xfe0f']
['0x2714', '0xfe0f']
['0x274c']


'0x1f44e'

In [46]:
@anvil.server.callable
def srvGenerateWordChain(quiet=True, startWord=None, strict=True):
  steps = generateChain(quiet=quiet, startWord=startWord, strict=strict)
  return steps

@anvil.server.callable
def srvValidateChain(startWord, endWord, wordChain):
  isOk, statuses = checkValidChain(startWord, endWord, wordChain)
  return isOk, statuses



In [ ]:
anvil.server.wait_forever()